In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
def my_formatter(x):
    if x < 0:
        return '{:,.0f}'.format(-x)
    return '{:,.0f}'.format(x)
pd.options.display.float_format = my_formatter

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='出貨明細')

In [ ]:
df.columns

In [ ]:
df['Group'].unique()

In [ ]:
cus_list = pd.read_clipboard()

In [ ]:
cus_classification = dict(zip(cus_list['Cus'], cus_list['Group']))

In [ ]:
laird2021 = df[(df['Group'] == 'Laird') & (df['預交日期'].between('2021-01-01', '2021-12-31')) & (df['狀態'].str.contains('出'))]

In [ ]:
laird2018 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2018.xlsx")

In [ ]:
laird2019 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2019.xlsx")

In [ ]:
laird2020 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2020.xlsx")

In [ ]:
raw_data = pd.concat([laird2018, laird2019, laird2020], axis=0)

In [ ]:
raw_data['數量'] = raw_data.apply(lambda x: x['數量'] * 1000 if x['單位'] == 'KPCS' else x['數量'], axis=1)

In [ ]:
raw_data['Group'] = raw_data['客戶名稱'].map(cus_classification)

In [ ]:
result = raw_data[(raw_data['負責業務'].isin(['楊婉芬', '沈思明', '鄭裡緗', '鄭里緗'])) & (raw_data['Group'] == 'Laird') & (raw_data['狀態'].str.contains('出'))]

In [ ]:
result['預交日期'] = pd.to_datetime(result['預交日期'], yearfirst=True)

In [ ]:
result['預交年份'] = result['預交日期'].dt.year

In [ ]:
result = result[['Group', '預交年份', '產品分類', '品名', '數量']]

In [ ]:
result2 = laird2021[['Group', '預交年份', '產品分類', '品名', '數量']]

In [ ]:
final = pd.concat([result, result2], axis=0)

In [ ]:
bead_final = final[~final['產品分類'].str.contains('MIP|WIP|MCI|AIP')]

In [ ]:
bead_final['Dimension'] = bead_final['品名'].str.extract('([0-9]+)')

In [ ]:
bead_final['Size'] = bead_final['Dimension'].map(lambda x: '0402/0603' if x == '1005' or x == '1608' else 'Others')

In [ ]:
answer = bead_final.groupby(['預交年份', 'Size'])[['數量']].sum().unstack('Size').reset_index()

In [ ]:
answer.columns = ['年份', '0402/0603', 'Others']

In [ ]:
answer.to_excel(r"C:\Users\kaihsu\Desktop\laird2018~2021出貨比例.xlsx", index=False)